## Homework 08:  Parallel Programming 01

## Due Date: Apr 12, 2023, 11:59pm

#### Firstname Lastname: Ching-Tsung(Deron) Tsai

#### E-mail: ct2840@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (50p)**

Write an MPI program `sol08pr01.py` that does the following for some arbitrary number of processes $N \geq 2$. Here the number of processes $N$ is given as `N` while calling the code `sol08pr01.py` as: 

`mpirun -n N python3 sol08pr01.py`


Every process will contain one buffer with one integer variable, each of which is initialized to $0$.

For $r=0, 1, \dots, N - 1$, Process $r$ updates its buffer to the value received by $r-1$ (this should only be done for $r \geq 1$), then it squares its rank $r$, adds the result $r^2$ to the value of its own buffer, and then sends the sum to Process $r + 1$. Note that for $r=N-1$ the result will be sent to Process $0$, i.e. by convention, Process $N$ is the same as Process $0$. At the end Process $0$ prints the received value. 

Provide results for: $N = 10$, $N = 15$, $N = 20$, $N = 25$. These are probably more than the available processes on your machine: you can use the option `--oversubscribe` in `mpirun` to let MPI run things anyway.



**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code. 



In [15]:
import numpy
x1 = numpy.zeros(1, dtype='i')
x1 + 2*2

array([4], dtype=int32)

In [1]:
%%writefile sol08pr01.py
import numpy
from mpi4py import MPI

comm = MPI.COMM_WORLD    # global communicator
N = comm.Get_size()     # num of process
rank = comm.Get_rank()   # current rank
data = numpy.array([0], dtype='i')   # initialize with 0; this is the buffer


if rank == 0:   # 1st node
    comm.Send([data, MPI.INT], dest=1)  
    comm.Recv([data, MPI.INT], source=(N-1))
    print(data)
elif rank == (N-1): # last node
    comm.Recv([data, MPI.INT], source=(rank-1)) # receive info from previous rank
    data = rank**2 + data
    comm.Send([data, MPI.INT], dest=0) 
else: 
    comm.Recv([data, MPI.INT], source=(rank-1)) # receive info from previous rank
    data = rank**2 + data                       # update info
    comm.Send([data, MPI.INT], dest=(rank+1))   # send to next dest
    
    

Overwriting sol08pr01.py


In [2]:
import os
for n in [10, 15, 20, 25]:
    print(f"Running with n={n}")
    os.system(f"mpiexec -n {n} python sol08pr01.py")

Running with n=10
[285]
Running with n=15
[1015]
Running with n=20
[2470]
Running with n=25
[4900]


---

**Problem 2 (50p)**

Write an MPI program that does the following. There are two processes 0 and 1 that have to exchange $T=10$ messages.  


Process 0 initially reads two float variables from the standard input, call them $x, y$, and must ensure $x \neq 0$ and $y \neq 0$. For example this can be done as:

```
import sys


for line in sys.stdin:
    x = float(line)        
    if x != 0.0:
        break
for line in sys.stdin:
    y = float(line)        
    if y != 0.0:
        break
```


Both Process 0 and Process 1 will carry main results in an element that is part of a process buffer and called $p$. The value in $p$ is initially set to $1$. 


Now the exchange of messages is as follows.


0. Message00: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

1. Message01: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

2. Message01: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

3. Message02: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.


etc.

8. Message08: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

9. Message09: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

Finally, Process 0 prints the value in $p$ as a final result. 


Write the code that implements the protocol above. Additionally, provide results for: $(x, y) = (2, 4)$, $(x, y) = (1, 3)$, $(x, y) = (5, 7)$ and $(x, y) = (5, 10)$.


**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code.



In [10]:
# create standard input file:
for (x,y) in [(2,4),(1,3),(5,7),(5,10)]:
    with open(f'input_{x}_{y}.txt', 'w') as f:
        f.write(f"{x}\n{y}\n")
# read the file using: python my_program.py < input.txt

In [16]:
%%writefile sol08pr02.py
import numpy
import sys
from mpi4py import MPI

comm = MPI.COMM_WORLD    # global communicator
rank = comm.Get_rank()   # current rank
T = 10 # 10 task

# MPI:
if rank == 0:
    # read standard inputs:
    for line in sys.stdin:
        x = float(line)        
        if x != 0.0:
            break                    # break when get the 1st stdin
    for line in sys.stdin:
        y = float(line)        
        if y != 0.0:
            break
    comm.send(y, dest=1, tag=100)    # send y to 2nd process
    p = 1                            # initiate value p as 1
    for i in range(T//2):            # 5 task in process 0
        p *= x  
        comm.send(p, dest=1)         # send message 00, 02, 04, 06, 08
        p = comm.recv(source=1)      # receive message 01, 03, 05, 07, 09
    print(f"Final output of p using input {x} & {y}: {p}")
elif rank == 1:
    y = comm.recv(source=0, tag=100)
    for i in range(T//2):            # 5 task in process 1
        p = comm.recv(source=0)      # receive message 00, 02, 04, 06, 08
        p /= y 
        comm.send(p, dest=0)         # send message 01, 03, 05, 07, 09

Overwriting sol08pr02.py


In [18]:
!mpiexec -n 2 python sol08pr02.py - < input_2_4.txt

Final output of p using input 2.0 & 4.0: 0.03125


In [19]:
!mpiexec -n 2 python sol08pr02.py - < input_1_3.txt

Final output of p using input 1.0 & 3.0: 0.004115226337448559


In [20]:
!mpiexec -n 2 python sol08pr02.py - < input_5_7.txt

Final output of p using input 5.0 & 7.0: 0.1859344320818706


In [21]:
!mpiexec -n 2 python sol08pr02.py - < input_5_10.txt

Final output of p using input 5.0 & 10.0: 0.03125
